# CaseStudy III
In this notebook, real-life MCDA problem will be solved using multiple modules. CaseStudy III group 30 alternatives into 4 clusters using Promethee Preference With Interactions.

## Definition of inputs and problem formalization

In [11]:
import pandas as pd
from core.enums import Direction, GeneralCriterion, InteractionType
from modular_parts.weights import calculate_srf_weights
from modular_parts.preference import (compute_preference_indices,
                                      compute_preference_indices_with_interactions)
from modular_parts.clustering import group_into_ordered_clusters

alternatives = ["UK", "SWE", "DEN", "NET", "SWI", "FIN", "NOR", "IRE", "AUS",
                "GER", "FRA",
                "BEL", "SPA", "EST", "ITA", "SLO", "POR", "HUN", "CZE", "SLK",
                "POL", "GRE",
                "LAT", "LIT", "ROM", "CRO", "BUL", "RUS", "UKR", "KAZ", "AZE"]
criteria = ['g1', 'g2', 'g3', 'g4', 'g5', 'g6']
criteria_directions = [Direction.MAX, Direction.MAX, Direction.MAX,
                       Direction.MAX, Direction.MAX, Direction.MAX]

generalised_criteria = [GeneralCriterion.V_SHAPE_INDIFFERENCE,
                        GeneralCriterion.V_SHAPE,
                        GeneralCriterion.V_SHAPE_INDIFFERENCE,
                        GeneralCriterion.V_SHAPE,
                        GeneralCriterion.LEVEL,
                        GeneralCriterion.V_SHAPE]
preference_thresholds = [5, 2, 5, 2, 0.9, 2]
indifference_thresholds = [1, None, 1, None, 0.3, None]
s_parameter = [None, None, None, None, 0.5, None]
alternatives_performances = [[94.3, 81.4, 78.5, 85.0, 16.4, 87.8],
                             [91.0, 86.7, 64.3, 81.5, 26.0, 80.2],
                             [94.7, 83.4, 64.0, 87.0, 18.5, 86.0],
                             [90.3, 85.3, 58.1, 87.0, 15.7, 82.4],
                             [89.0, 93.3, 53.9, 85.0, 12.6, 81.9],
                             [89.7, 61.8, 67.2, 85.0, 21.8, 85.2],
                             [82.6, 71.6, 63.3, 85.0, 10.8, 88.7],
                             [95.3, 54.5, 75.5, 81.5, 12.2, 84.7],
                             [88.3, 61.2, 54.9, 85.0, 15.3, 77.8],
                             [89.0, 62.0, 57.5, 85.0, 14.1, 70.8],
                             [83.3, 48.7, 58.7, 83.5, 12.4, 73.4],
                             [89.7, 46.1, 58.9, 88.5, 10.0, 80.9],
                             [88.3, 31.2, 61.2, 74.5, 3.9, 70.1],
                             [81.0, 45.4, 53.5, 69.5, 1.5, 69.8],
                             [73.3, 37.3, 61.2, 71.0, 4.1, 69.7],
                             [68.7, 38.1, 63.5, 73.0, 4.4, 66.3],
                             [87.0, 19.0, 56.8, 71.0, 1.6, 72.4],
                             [80.3, 17.0, 59.7, 71.0, 5.8, 58.1],
                             [75.3, 27.2, 54.7, 67.5, 4.0, 58.1],
                             [76.3, 30.0, 52.5, 67.5, 1.6, 54.8],
                             [74.6, 22.1, 54.3, 70.0, 1.3, 59.5],
                             [74.0, 11.3, 61.0, 67.5, 1.7, 60.8],
                             [71.0, 23.5, 56.2, 65.5, 1.0, 55.3],
                             [66.3, 19.1, 56.7, 67.5, 1.2, 55.3],
                             [67.0, 12.1, 48.2, 56.0, 0.6, 52.1],
                             [55.6, 3.2, 51.9, 62.0, 2.4, 54.6],
                             [61.3, 6.8, 47.4, 56.0, 1.1, 49.3],
                             [46.9, 10.6, 55.5, 38.5, 1.9, 36.6],
                             [40.2, 4.0, 49.0, 46.0, 1.4, 31.4],
                             [49.6, 3.5, 32.3, 42.0, 0.5, 43.4],
                             [40.9, 3.3, 21.4, 38.0, 0.4, 43.6],
                             ]

criteria_directions = pd.Series(criteria_directions, criteria)
generalised_criteria = pd.Series(generalised_criteria, criteria)
preference_thresholds = pd.Series(preference_thresholds, criteria)
indifference_thresholds = pd.Series(indifference_thresholds, criteria)
s_parameters = pd.Series(data=None, index=criteria)
criteria_ranks = pd.Series({
    'g1': 8, 'g2': 4, 'g3': 7, 'g4': 2, 'g5': 4, 'g6': 1
})
criteria_weight_ratio = 5
alternatives_performances = pd.DataFrame(data=alternatives_performances,
                                         index=alternatives, columns=criteria)

alternatives_performances

C:\Users\teba_sgrabowski\AppData\Local\Temp\ipykernel_7592\2586806607.py:63: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s_parameters = pd.Series(data=None, index=criteria)


,g1,g2,g3,g4,g5,g6
UK,94.3,81.4,78.5,85.0,16.4,87.8
SWE,91.0,86.7,64.3,81.5,26.0,80.2
DEN,94.7,83.4,64.0,87.0,18.5,86.0
NET,90.3,85.3,58.1,87.0,15.7,82.4
SWI,89.0,93.3,53.9,85.0,12.6,81.9
FIN,89.7,61.8,67.2,85.0,21.8,85.2
NOR,82.6,71.6,63.3,85.0,10.8,88.7
IRE,95.3,54.5,75.5,81.5,12.2,84.7
AUS,88.3,61.2,54.9,85.0,15.3,77.8
GER,89.0,62.0,57.5,85.0,14.1,70.8


Here are interactions used in Promethee Preference With Interactions (M5).

In [12]:
interactions = pd.DataFrame(data=[['g1', 'g5', InteractionType.STN, 0.03],
                                  ['g2', 'g6', InteractionType.WKN, -0.02]],
                            columns=['criterion_1', 'criterion_2', 'type',
                                     'coefficient'])
interactions

,criterion_1,criterion_2,type,coefficient
0,g1,g5,InteractionType.STN,0.03
1,g2,g6,InteractionType.WKN,-0.02


## Modules usage

Weights calculated with SRF method.

In [13]:
criteria_weights = calculate_srf_weights(criteria_ranks=criteria_ranks,
                                         criteria_weight_ratio=criteria_weight_ratio,
                                         decimal_place=2)
criteria_weights

g1    28.70
g2    15.55
g3    25.43
g4     9.01
g5    15.56
g6     5.74
dtype: float64

The inclusion of "Promethee Preference" here is solely for the purpose of comparison with "Preference With Interactions". It is included to allow user of PyMCDA to easily recognize the difference in results.

In [14]:
preference_indices, partial_preference_indices = compute_preference_indices(
    alternatives_performances, preference_thresholds, indifference_thresholds,
    s_parameters, generalised_criteria,
    criteria_directions, criteria_weights, decimal_place=6,)

In [15]:
preference_indices

,UK,SWE,DEN,NET,SWI,FIN,NOR,IRE,AUS,GER,...,GRE,LAT,LIT,ROM,CRO,BUL,RUS,UKR,KAZ,AZE
UK,0.000000,0.566882,0.305991,0.604810,0.754375,0.725573,0.852485,0.585809,0.909891,0.909891,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
SWE,0.311131,0.000000,0.311131,0.518802,0.481698,0.332658,0.598160,0.311131,0.744849,0.694619,...,0.891912,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
DEN,0.401240,0.341259,0.000000,0.711321,0.844484,0.555616,0.688269,0.438554,1.000000,1.000000,...,0.872837,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
NET,0.245625,0.147515,0.147740,0.000000,0.485064,0.245625,0.688269,0.401240,0.592474,0.480173,...,0.745675,0.802898,0.771107,1.000000,1.000000,1.000000,0.847405,1.000000,1.000000,1.000000
SWI,0.155516,0.294419,0.155516,0.155516,0.000000,0.155516,0.598160,0.323432,0.212921,0.212921,...,0.745675,0.745675,0.745675,1.000000,0.809256,1.000000,0.745675,0.993642,1.000000,1.000000
FIN,0.155616,0.268319,0.295495,0.467347,0.467347,0.000000,0.627030,0.415592,0.542704,0.467347,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
NOR,0.025833,0.147515,0.057406,0.311731,0.311731,0.212921,0.000000,0.303030,0.467247,0.467247,...,0.828330,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
IRE,0.000000,0.548530,0.254325,0.598760,0.598760,0.541354,0.696970,0.000000,0.598760,0.598760,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
AUS,0.000000,0.090109,0.000000,0.000000,0.155616,0.000000,0.442644,0.401240,0.000000,0.213021,...,0.745675,0.745675,0.745675,1.000000,0.872837,1.000000,0.745675,1.000000,1.000000,1.000000
GER,0.000000,0.090109,0.000000,0.000000,0.320927,0.015552,0.442644,0.401240,0.163936,0.000000,...,0.745675,0.764749,0.745675,1.000000,1.000000,1.000000,0.809256,1.000000,1.000000,1.000000


Calculating Promethee Preference With Interactions.

In [16]:
preference_indices_int, partial_preference_indices_int = compute_preference_indices_with_interactions(
    alternatives_performances, criteria_weights, preference_thresholds,
    indifference_thresholds, pd.Series(data=None, index=criteria),
    generalised_criteria, criteria_directions, interactions, decimal_place=6)

C:\Users\teba_sgrabowski\AppData\Local\Temp\ipykernel_7592\804971765.py:3: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  indifference_thresholds, pd.Series(data=None, index=criteria),


In [17]:
partial_preference_indices_int

UK    SWE  DEN   NET    SWI    FIN  NOR  IRE    AUS    GER  \
criteria                                                                    
g1       UK   0.0  0.575  0.0  0.75  1.000  0.900  1.0  0.0  1.000  1.000   
         SWE  0.0  0.000  0.0  0.00  0.250  0.075  1.0  0.0  0.425  0.250   
         DEN  0.0  0.675  0.0  0.85  1.000  1.000  1.0  0.0  1.000  1.000   
         NET  0.0  0.000  0.0  0.00  0.075  0.000  1.0  0.0  0.250  0.075   
         SWI  0.0  0.000  0.0  0.00  0.000  0.000  1.0  0.0  0.000  0.000   
...           ...    ...  ...   ...    ...    ...  ...  ...    ...    ...   
g6       BUL  0.0  0.000  0.0  0.00  0.000  0.000  0.0  0.0  0.000  0.000   
         RUS  0.0  0.000  0.0  0.00  0.000  0.000  0.0  0.0  0.000  0.000   
         UKR  0.0  0.000  0.0  0.00  0.000  0.000  0.0  0.0  0.000  0.000   
         KAZ  0.0  0.000  0.0  0.00  0.000  0.000  0.0  0.0  0.000  0.000   
         AZE  0.0  0.000  0.0  0.00  0.000  0.000  0.0  0.0  0.000  0.000   

              ...  GRE  LAT  LIT  ROM  CRO  BUL  RUS  UKR  KAZ  AZE  
criteria      ...                                                    
g1       UK   ...  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  
         SWE  ...  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  
         DEN  ...  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  
         NET  ...  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  
         SWI  ...  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  
...           ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  
g6       BUL  ...  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0  1.0  
         RUS  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  
         UKR  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
         KAZ  ...  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0  
         AZE  ...  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  0.1  0.0  

[186 rows x 31 columns]

In [18]:
preference_indices_int

,UK,SWE,DEN,NET,SWI,FIN,NOR,IRE,AUS,GER,...,GRE,LAT,LIT,ROM,CRO,BUL,RUS,UKR,KAZ,AZE
UK,0.000000,0.566882,0.305991,0.609207,0.761530,0.719972,0.856782,0.577356,0.910783,0.910783,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
SWE,0.311131,0.000000,0.311131,0.518802,0.485556,0.334156,0.609864,0.311131,0.742986,0.690754,...,0.892982,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
DEN,0.401240,0.341259,0.000000,0.718499,0.849014,0.552032,0.697348,0.431159,1.000000,1.000000,...,0.874096,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
NET,0.245625,0.147515,0.147740,0.000000,0.486220,0.245625,0.697348,0.401240,0.585743,0.470779,...,0.748193,0.804849,0.773373,1.000000,1.000000,1.000000,0.848916,1.000000,1.000000,1.000000
SWI,0.155516,0.282217,0.155516,0.155516,0.000000,0.155516,0.609864,0.323432,0.196858,0.196858,...,0.748193,0.748193,0.748193,1.000000,0.811144,1.000000,0.748193,0.993705,1.000000,1.000000
FIN,0.155616,0.268319,0.295495,0.467347,0.467347,0.000000,0.637893,0.412655,0.541328,0.467347,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
NOR,0.025833,0.147515,0.057406,0.311731,0.311731,0.196858,0.000000,0.288806,0.456374,0.456374,...,0.830030,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
IRE,0.000000,0.548530,0.254325,0.598760,0.598760,0.541354,0.705796,0.000000,0.598760,0.598760,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
AUS,0.000000,0.090109,0.000000,0.000000,0.155616,0.000000,0.458878,0.401240,0.000000,0.213021,...,0.748193,0.748193,0.748193,1.000000,0.874096,1.000000,0.748193,1.000000,1.000000,1.000000
GER,0.000000,0.090109,0.000000,0.000000,0.320927,0.015552,0.458878,0.401240,0.163936,0.000000,...,0.748193,0.767078,0.748193,1.000000,1.000000,1.000000,0.811144,1.000000,1.000000,1.000000


Finally, alternatives are clustered into 4 groups using PrometheeCluster.

In [19]:
cluster = group_into_ordered_clusters(preference_indices_int, 4)
cluster

C1                        [UK, SWE, DEN, NET, FIN, IRE]
C2    [SWI, NOR, AUS, GER, FRA, BEL, SPA, ITA, SLO, ...
C3        [EST, POR, CZE, SLK, POL, GRE, LAT, LIT, RUS]
C4                       [ROM, CRO, BUL, UKR, KAZ, AZE]
Name: Alternatives in clusters, dtype: object

In [20]:
from core.clusters_commons import change_clusters_output

change_clusters_output(cluster)

AUS    C2
AZE    C4
BEL    C2
BUL    C4
CRO    C4
CZE    C3
DEN    C1
EST    C3
FIN    C1
FRA    C2
GER    C2
GRE    C3
HUN    C2
IRE    C1
ITA    C2
KAZ    C4
LAT    C3
LIT    C3
NET    C1
NOR    C2
POL    C3
POR    C3
ROM    C4
RUS    C3
SLK    C3
SLO    C2
SPA    C2
SWE    C1
SWI    C2
UK     C1
UKR    C4
dtype: object